In [1]:
%%capture
!pip install -r https://raw.githubusercontent.com/jpcosta90/once-learning/refs/heads/main/scripts/requirements.txt

In [2]:
!llama model list

+----------------------------------+------------------------------------------+----------------+
| Model Descriptor                 | Hugging Face Repo                        | Context Length |
+----------------------------------+------------------------------------------+----------------+
| Llama3.1-8B                      | meta-llama/Llama-3.1-8B                  | 128K           |
+----------------------------------+------------------------------------------+----------------+
| Llama3.1-70B                     | meta-llama/Llama-3.1-70B                 | 128K           |
+----------------------------------+------------------------------------------+----------------+
| Llama3.1-405B:bf16-mp8           | meta-llama/Llama-3.1-405B                | 128K           |
+----------------------------------+------------------------------------------+----------------+
| Llama3.1-405B                    | meta-llama/Llama-3.1-405B-FP8            | 128K           |
+-----------------------------

In [24]:
from huggingface_hub import snapshot_download
import os

# Get the token from the environment variable
hf_token = os.getenv("HF_TOKEN")

In [4]:
import torch
print(torch.cuda.is_available())  # Should return True if CUDA is available

True


In [5]:
import requests
import re
import torch
from PIL import Image
from transformers import MllamaForConditionalGeneration, AutoProcessor
from huggingface_hub import InferenceClient
from transformers import pipeline

In [6]:
%%time

model_id = "meta-llama/Llama-3.2-11B-Vision-Instruct"

model = MllamaForConditionalGeneration.from_pretrained(
    model_id,
    load_in_4bit=True,
)
processor = AutoProcessor.from_pretrained(model_id)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

CPU times: user 25.2 s, sys: 22 s, total: 47.2 s
Wall time: 2min 5s


In [7]:
import pandas as pd

# Carregar o dataframe
df = pd.read_csv('https://raw.githubusercontent.com/jpcosta90/once-learning/refs/heads/main/data/one-page-document/one_page_document_complarison_sample.csv')
df = df[['file1','file2','same_pattern']]

In [19]:
# Definir a função de comparação
def compare_documents(file1, file2):
    try:
        import requests
        from PIL import Image
        import re

        # URLs base das imagens
        base_url = 'https://raw.githubusercontent.com/jpcosta90/once-learning/main/files/'

        # Carregar as imagens
        reference_image_url = base_url + file1
        image_to_classify_url = base_url + file2

        reference_image = Image.open(requests.get(reference_image_url, stream=True).raw)
        image_to_classify = Image.open(requests.get(image_to_classify_url, stream=True).raw)

        # Preparar as mensagens com o prompt ajustado
        messages = [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": '''
                    Respond with a JSON object with elements status: (standard or out of standard) and justification. Compare two images of company documents, evaluating whether the second document conforms to the standard of the first one in terms of content and layout.

                    - **Image 1**: Standard document.
                    - **Image 2**: Document to be compared.

                    Respond only with a JSON object.

                    Do not include any additional text.
                    '''},
                    {"type": "image"},  # Imagem de referência
                    {"type": "image"},  # Imagem a ser analisada
                ]
            }
        ]

        # Aplicar o template do chat sem adicionar o prompt de geração
        input_text = processor.apply_chat_template(messages, add_generation_prompt=False)

        # Pré-processar as imagens e o texto
        inputs = processor(
            images=[reference_image, image_to_classify],
            text=input_text,
            return_tensors="pt"
        ).to(model.device)

        # Gerar a resposta
        output = model.generate(**inputs, max_new_tokens=200)

        # Decodificar a saída e pular tokens especiais
        decoded_output = processor.decode(output[0], skip_special_tokens=True)

        # Extrair apenas a resposta do assistente
        assistant_response = decoded_output.strip()

        # Usar expressão regular para extrair o objeto JSON
        match = re.search(r'\{.*?\}', assistant_response, re.DOTALL)
        if match:
            assistant_response = match.group(0)
        else:
            assistant_response = "No valid JSON response."

        return assistant_response
    except Exception as e:
        return f"Error: {str(e)}"

In [20]:
# Aplicar a função ao dataframe
from tqdm import tqdm
tqdm.pandas()

df['llama32_response'] = df.progress_apply(
    lambda row: compare_documents(row['file1'], row['file2']), axis=1
)

100%|██████████| 20/20 [07:21<00:00, 22.06s/it]


In [26]:
# Salvar o dataframe atualizado
df.to_csv('llama_result_sample.csv', index=False)

In [25]:
import os

# Get the token from the environment variable
gh_token = os.getenv("GH_TOKEN")

In [27]:
import requests
import base64

# Leitura do arquivo CSV
with open('llama_result_sample.csv', 'rb') as f:
    content = f.read()

# Codificação do conteúdo em base64
content_b64 = base64.b64encode(content).decode('utf-8')

# Configurar os parâmetros da API
repo = 'jpcosta90/once-learning'
path = 'results/llama_result_sample.csv'
message = 'Adiciona sample_comparisons.csv'
token = gh_token

In [29]:
# URL da API do GitHub
url = f'https://api.github.com/repos/{repo}/contents/{path}'

# Cabeçalhos da requisição
headers = {
    'Authorization': f'Bearer {token}',
    'Content-Type': 'application/json'
}

# Dados da requisição
data = {
    'message': message,
    'content': content_b64
}

# Enviar a requisição PUT
response = requests.put(url, headers=headers, json=data)

# Verificar a resposta
if response.status_code == 201:
    print('Arquivo enviado com sucesso!')
else:
    print(f'Erro ao enviar o arquivo: {response.content}')

Erro ao enviar o arquivo: b'{"message":"Bad credentials","documentation_url":"https://docs.github.com/rest","status":"401"}'


In [30]:
# Verificar autenticação do token
response = requests.get('https://api.github.com/user', headers=headers)
if response.status_code == 200:
    print('Autenticação bem-sucedida!')
    print(f"Usuário autenticado: {response.json()['login']}")
else:
    print(f"Erro na autenticação: {response.content}")

Erro na autenticação: b'{"message":"Bad credentials","documentation_url":"https://docs.github.com/rest","status":"401"}'
